In [2]:
!pip install tensorflow
import pandas as pd
import tensorflow as tf
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score
df= pd.read_csv("Reviews(13).csv")
df

,review,Label
0,I do not know how you can say he is a doctor,Negative
1,He is not a doctor. Just know how to charge money,negative
2,Alhmdulillah positive experience,positive
3,Glad to have good experience,positive
4,Fair review.,Neutral
...,...,...
24430,I booked an appointment for my friend for 12:...,Negative
24431,Not treated in a good way,Negative
24432,Doctor Noor Samad khan was a great doctor and ...,Positive
24433,I feel he is not competent in the field of sex...,Negative


In [3]:
!pip install torch
import numpy as np
import torch


In [4]:
## Correcting the spelling of one misspelled label
df['Label'] = df['Label'].str.replace('negative', 'Negative')
df['Label'] = df['Label'].str.replace('neutral', 'Neutral')
df['Label'] = df['Label'].str.replace('positive', 'Positive')
df['Label'] = df['Label'].str.replace('Netural', 'Neutral')
df['Label'] = df['Label'].str.replace(' Neutral', 'Neutral')

df['Label'].unique()

df['Label'].value_counts()

Positive    11730
Negative     9109
Neutral      3596
Name: Label, dtype: int64

In [5]:
#loading the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#Convert each review text into a sequence of tokens that can be input into the BERT model.
X = df['review'].apply(lambda x: tokenizer.encode_plus(x, add_special_tokens=True, return_attention_mask=True, return_token_type_ids=True, padding='max_length', truncation=True, max_length=256))
y = df['Label'].map({'Positive': 2, 'Neutral': 1, 'Negative': 0})

In [6]:
# Prepare the data
#split the pre-processed data into training and testing sets
train_size = int(0.8 * len(X))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

In [7]:
# Convert the data to PyTorch tensors
train_input_ids = torch.tensor([x['input_ids'] for x in train_X])
train_attention_masks = torch.tensor([x['attention_mask'] for x in train_X])
train_token_type_ids = torch.tensor([x['token_type_ids'] for x in train_X])
train_y = torch.tensor(train_y.values)

test_input_ids = torch.tensor([x['input_ids'] for x in test_X])
test_attention_masks = torch.tensor([x['attention_mask'] for x in test_X])
test_token_type_ids = torch.tensor([x['token_type_ids'] for x in test_X])
test_y = torch.tensor(test_y.values)


In [8]:
# Create the PyTorch DataLoader
train_dataset = torch.utils.data.TensorDataset(train_input_ids, train_attention_masks, train_token_type_ids, train_y)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=16)

test_dataset = torch.utils.data.TensorDataset(test_input_ids, test_attention_masks, test_token_type_ids, test_y)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16)

In [9]:
# Create the BERT model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)



Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [11]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

#Adam optimizer to optimize the BERT model during training
optimizer = torch.optim.Adam(model.parameters(), lr=2e-5, eps=1e-8)
#Cross-Entropy Loss object to calculate the loss between predicted labels and actual labels during training.
criterion = torch.nn.CrossEntropyLoss()

#Compute the gradients of the loss wrt model parameters & update model weights & biases
for epoch in range(4):
    train_loss = 0.0
    model.train()
    for batch in train_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, token_type_ids, labels = batch
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
#Accumulates the training loss over each batch of data
        train_loss += loss.item() * input_ids.size(0)
    train_loss = train_loss / len(train_dataloader.dataset)
    print('Epoch: {} \tTraining Loss: {:.6f}'.format(epoch + 1, train_loss))


Epoch: 1 	Training Loss: 0.191683
Epoch: 2 	Training Loss: 0.087171
Epoch: 3 	Training Loss: 0.063569
Epoch: 4 	Training Loss: 0.046668


In [12]:
# Evaluate the model
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, token_type_ids, labels = batch
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        _, predicted = torch.max(outputs.logits, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(predicted.tolist())

# Calculate accuracy and F1 score
accuracy = accuracy_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred, average='weighted')
print('Accuracy: {:.2f}%'.format(accuracy * 100))
print('F1 Score: {:.2f}%'.format(f1 * 100))


Accuracy: 97.24%
F1 Score: 97.25%


In [13]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score

report = classification_report(y_true, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1843
           1       0.92      0.93      0.93       707
           2       0.99      0.98      0.99      2337

    accuracy                           0.97      4887
   macro avg       0.96      0.96      0.96      4887
weighted avg       0.97      0.97      0.97      4887



In [14]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder and fit it to the sentiment classes
label_encoder = LabelEncoder()
label_encoder.fit(['Positive', 'Negative', 'Neutral'])

# Define a function to predict the sentiment of a given text
def predict_sentiment(text):
    encoded_review = tokenizer.encode_plus(
        text,
        add_special_tokens=True,
        pad_to_max_length=True,
        return_attention_mask=True,
        return_token_type_ids=True,
        return_tensors='pt'
    )
    
    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    token_type_ids = encoded_review['token_type_ids'].to(device)
    
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        _, predicted = torch.max(outputs.logits, 1)
        sentiment = label_encoder.inverse_transform(predicted.tolist())[0]
        return sentiment

# Example usage
print(predict_sentiment("Good doctor!"))
print(predict_sentiment("This is the worst hospital ever."))
print(predict_sentiment("Average experience tha"))
print(predict_sentiment("Bohat ajeeb"))
print(predict_sentiment("I had to wait a lot for doctor but doctor treated me well"))


Positive


C:\Users\Admin\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2339: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Negative
Neutral
Negative
Neutral


In [16]:
torch.save(model.state_dict(), 'bert_model.pth')


In [18]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
model.load_state_dict(torch.load('bert_model.pth'))
model.eval()
